<a href="https://colab.research.google.com/github/Shai670/Residential_consumption_modeling/blob/main/Battery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from IPython.display import clear_output 

import matplotlib.pyplot as plt

In [3]:
!git clone https://github.com/Shai670/Residential_consumption_modeling
clear_output()

In [4]:
pv = pd.read_csv('/content/Residential_consumption_modeling/DATA/PV_1.5.csv', index_col='time')
pv.index = pd.to_datetime(pv.index)
pv

,AC
time,
2021-01-01 06:00:00,-0.450000
2021-01-01 07:00:00,-0.450000
2021-01-01 08:00:00,-0.450000
2021-01-01 09:00:00,101.638826
2021-01-01 10:00:00,355.759449
...,...
2022-01-01 01:00:00,-0.450000
2022-01-01 02:00:00,-0.450000
2022-01-01 03:00:00,-0.450000


In [6]:
load = pd.DataFrame(index=pd.date_range(start='2021-01-01 00:00:00', end='2021-12-31 23:59:00', freq='MIN'))
load['load'] = np.random.rand(len(load))
load

,load
2021-01-01 00:00:00,0.424173
2021-01-01 00:01:00,0.121527
2021-01-01 00:02:00,0.466699
2021-01-01 00:03:00,0.865463
2021-01-01 00:04:00,0.623134
...,...
2021-12-31 23:55:00,0.134559
2021-12-31 23:56:00,0.811116
2021-12-31 23:57:00,0.579752
2021-12-31 23:58:00,0.892563


In [14]:
class Pribor:
  """
  Пример класса для электроприбора
  """

  def __init__(self, Pnom, capacity,Pgen,Pload):
    self.Pnom = Pnom # номинальная мощность батареи
    self.capacity = capacity # ёмкость 
    self.Pgen = Pgen
    self.Pload = Pload
    self.soc=0

    def working (self, Pnom, capacity, Pgen, Pload,SOC,SOCmax):
    
     if Pgen>Pload : print( SOC>=SOCmax)



    